We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [19]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import (
    Grad
)  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import General_Functions as genfunc
import numpy as np
import scipy.sparse as sp
import time
T_MAX: float = 100  # The maximum time for the simulation
ALPHA: float = 5  # Dissipative constant in the LLG equation.
THETA: float = 0.505  # Should be strictly above 1/2 for unconditional stability
K: float = 0.01  # TIME STEP
KAPPA: float = 1.0  # Determines the relative strength of the elastic vs. magnetic parts.
mu: float = 1  # first lame constant
lam: float = 1  # second lame constant
lambda_m: float = 1  # saturation magnetostrain parameter
H_MAX: float = 0.5  # Determines how fine the mesh should be.

In [20]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left = Plane(Pnt(0, 0, 0), Vec(-1, 0, 0)).bc("left")
    right = Plane(Pnt(1, 1, 1), Vec(1, 0, 0)).bc("right")
    front = Plane(Pnt(0, 0, 0), Vec(0, -1, 0)).bc("front")
    back = Plane(Pnt(1, 1, 1), Vec(0, 1, 0)).bc("back")
    bot = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("bot")
    top = Plane(Pnt(1, 1, 1), Vec(0, 0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add(cube)
    # cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry


ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [21]:
fes_mag = VectorH1(
    mesh, order=1
)  # the finite element space for the magnetisation m_h^i
fes_eps_m = MatrixValued(
    H1(mesh, order=1), dim=3
)  # matrix FE space on the magnetic part
fes_disp = VectorH1(
    mesh, order=1, dirichlet="bot"
)  # the finite element space for the displacement u_h^i

# print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)
prev_disp_gfu = GridFunction(fes_disp) #  used to store the previous displacement
# body force and traction force
body_factor = Parameter(-1.0)
f_body = CoefficientFunction( (0, 0, body_factor) )
surface_factor = Parameter(1)
g_surface = CoefficientFunction([(0,0,surface_factor) if bc=="left" else 0 for bc in mesh.GetBoundaries()])

In [22]:
# Initial conditions
mag_gfu = magfunc.give_random_magnetisation(mag_gfu)
disp_gfu = elfunc.give_random_displacement(disp_gfu)
velocity_gfu = GridFunction(fes_disp)  # velocity
velocity_gfu = elfunc.give_random_displacement(velocity_gfu)  # An initial velocity. Should only be used once in iteration.
Draw(mag_gfu)
magfunc.magnetic_energy(mag_gfu, mesh)

14.060980235944102

In [23]:
genfunc.export_to_vtk_file(disp_gfu, mag_gfu, mesh, export=False, index=0)

In [24]:
num_steps = genfunc.ceiling_division(T_MAX, K)  # This is ceiling division, using upside-down floor division.
#first run
proj_mag = magfunc.nodal_projection(mag_gfu, fes_mag)
strain_m = magfunc.build_strain_m(proj_mag, lambda_m)

mag_gfu = magfunc.update_magnetisation(fes_mag, mag_gfu, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m)
# subsequence runs
for i in range(1,num_steps):
    mag_gfu = magfunc.update_magnetisation(fes_mag, mag_gfu, ALPHA, THETA, K, KAPPA, disp_gfu, mu, lam, lambda_m)
    Redraw()
    print(f"Step {i}:integral |m*m|dx = {Integrate(mag_gfu*mag_gfu, mesh, VOL)} (should be 1)\n\
        energy      = {magfunc.magnetic_energy(mag_gfu, mesh)}")
    genfunc.export_to_vtk_file(disp_gfu, mag_gfu, mesh, export=True, index=i)

gmres completed in 0.004019737243652344, info=0, residual = 5.519769086248516e-09
gmres completed in 0.003988504409790039, info=0, residual = 1.909592928228676e-09
Step 1:integral |m*m|dx = 0.4253155187945218 (should be 1)
        energy      = 12.604165742916823
gmres completed in 0.0029909610748291016, info=0, residual = 4.736371850277976e-09
Step 2:integral |m*m|dx = 0.4385926691507804 (should be 1)
        energy      = 11.914569160743097
gmres completed in 0.004024028778076172, info=0, residual = 4.0260630385091645e-09
Step 3:integral |m*m|dx = 0.45160435689661915 (should be 1)
        energy      = 11.267707122370055
gmres completed in 0.003989458084106445, info=0, residual = 1.033040608611202e-08
Step 4:integral |m*m|dx = 0.4641588043358649 (should be 1)
        energy      = 10.672927463423381
gmres completed in 0.003961801528930664, info=0, residual = 2.132006349242488e-09
Step 5:integral |m*m|dx = 0.4761317734718564 (should be 1)
        energy      = 10.13445932712631
gmres 